# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [5]:
class Matrix:
    def __init__(self, n=None, m=None, values=None):
        if values is not None:
            self.data = values
            self.n = len(values)
            self.m = len(values[0]) if self.n > 0 else 0
            for row in values:
                if len(row) != self.m:
                    raise ValueError("All rows must have the same number of columns.")
        elif n is not None and m is not None:
            self.data = [[0 for _ in range(m)] for _ in range(n)]
            self.n = n
            self.m = m
        else:
            raise ValueError("Must provide either dimensions or values.")

    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        return Matrix(values=[[self.data[j][i] for j in range(self.n)] for i in range(self.m)])

    def row(self, n):
        return Matrix(values=[self.data[n]])

    def column(self, n):
        return Matrix(values=[[self.data[i][n]] for i in range(self.n)])

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        return Matrix(values=[row[m_0:m_1] for row in self.data[n_0:n_1]])

    def __getitem__(self, index):
        if isinstance(index, tuple):
            return self.data[index[0]][index[1]]
        return self.data[index]

    def __setitem__(self, index, value):
        self.data[index[0]][index[1]] = value

    def __eq__(self, other):
        return self.equals(other)

    def equals(self, other):
        if self.shape() != other.shape():
            return False
        for i in range(self.n):
            for j in range(self.m):
                if self.data[i][j] != other.data[i][j]:
                    return False
        return True

    def scalarmul(self, c):
        return Matrix(values=[[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape to add.")
        return Matrix(values=[[self.data[i][j] + other.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape to subtract.")
        return Matrix(values=[[self.data[i][j] - other.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, other):
        if self.m != other.n:
            raise ValueError("Incompatible dimensions for matrix multiplication.")
        return Matrix(values=[[sum(self.data[i][k] * other.data[k][j] for k in range(self.m)) for j in range(other.m)] for i in range(self.n)])

    def element_mult(self, other):
        if self.shape() != other.shape():
            raise ValueError("Matrices must have the same shape for element-wise multiplication.")
        return Matrix(values=[[self.data[i][j] * other.data[i][j] for j in range(self.m)] for i in range(self.n)])

def constant(n, m, c):
    return Matrix(values=[[c for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    return Matrix(n, m)

def ones(n, m):
    return Matrix(values=[[1 for _ in range(m)] for _ in range(n)])

def eye(n):
    return Matrix(values=[[1 if i == j else 0 for j in range(n)] for i in range(n)])

# Operator Overloading
def __mul__(self, other):
    if isinstance(other, (int, float)):
        return self.scalarmul(other)
    return self.mat_mult(other)

def __rmul__(self, other):
    return self * other

def __add__(self, other):
    return self.add(other)

def __sub__(self, other):
    return self.sub(other)

Matrix.__mul__ = __mul__
Matrix.__rmul__ = __rmul__
Matrix.__add__ = __add__
Matrix.__sub__ = __sub__

#examples
A = Matrix(values=[[1, 2], [3, 4]])
B = Matrix(values=[[5, 6], [7, 8]])
C = Matrix(values=[[1, 0], [0, 1]])

#properties
AB_C = A.mat_mult(B).mat_mult(C)
A_B_C = A.mat_mult(B.add(C))
print("AB*C == A*(B+C):", AB_C == A_B_C)

AB_not_BA = A.mat_mult(B) != B.mat_mult(A)
print("AB != BA:", AB_not_BA)

AI = A.mat_mult(eye(2))
print("A*I == A:", AI == A)

AB*C == A*(B+C): False
AB != BA: True
A*I == A: True
